## Part 2. Model Training & Evaluation - RNN

Now with the pretrained word embeddings acquired from Part 1 and the dataset acquired from Part
0, you need to train a deep learning model for topic classification using the training set, conforming
to these requirements:
- Use the pretrained word embeddings from Part 1 as inputs, together with your implementation
in mitigating the influence of OOV words; make them learnable parameters during training
(they are updated).
- Design a simple recurrent neural network (RNN), taking the input word embeddings, and
predicting a topic label for each sentence. To do that, you need to consider how to aggregate
the word representations to represent a sentence.
- Use the validation set to gauge the performance of the model for each epoch during training.
You are required to use accuracy as the performance metric during validation and evaluation.
- Use the mini-batch strategy during training. You may choose any preferred optimizer (e.g.,
SGD, Adagrad, Adam, RMSprop). Be careful when you choose your initial learning rate and
mini-batch size. (You should use the validation set to determine the optimal configuration.)
Train the model until the accuracy score on the validation set is not increasing for a few
epochs.
- Try different regularization techniques to mitigate overfitting.
- Evaluate your trained model on the test dataset, observing the accuracy score.

In [1]:
import json
import numpy as np
import random
import os
import lightning as L
from pathlib import Path
from torchtext import data, datasets
from utils.config import Config
from utils.train import train_rnn_model_with_parameters, train_rnn_model_with_parameters_regularzition
from utils.helper import SentenceDataset, collate_fn
from utils.analytics import load_tensorboard_logs
from models.RNN import RNN, RNNClassifier
from torch.utils.data import Dataset, DataLoader
from lightning.pytorch.callbacks import EarlyStopping, ModelCheckpoint

/home/linnsheng/Desktop/NTU/S3/Y1/NLP/SC4002/.venv/lib/python3.13/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
TEXT = data.Field(tokenize = 'spacy', tokenizer_language='en_core_web_sm', include_lengths=True)
LABEL = data.LabelField()

train_data, test_data = datasets.TREC.splits(TEXT, LABEL, fine_grained=False)

In [3]:
train_data, valid_data = train_data.split(random_state=random.seed(Config.SEED), split_ratio=0.8)

In [4]:
TEXT.build_vocab(train_data, vectors="glove.6B.300d")
LABEL.build_vocab(train_data)

### Import the embedding matrix and vocab index mapping (train data)

In [5]:
embedding_path = Path("models/embedding_matrix.npy")
index_from_word_path = Path("models/index_from_word.json")

embedding_matrix = np.load(embedding_path)
with index_from_word_path.open() as f:
    index_from_word = json.load(f)

### Dataset

In [6]:
train_dataset = SentenceDataset(train_data.examples, index_from_word, LABEL.vocab)
valid_dataset = SentenceDataset(valid_data.examples, index_from_word, LABEL.vocab)
test_dataset = SentenceDataset(test_data.examples, index_from_word, LABEL.vocab)        

### Train RNN Model

In [7]:
SEARCH_SPACE = {
    "batch_size": [32, 64, 128],
    "optimizer_name": ["RMSprop", "Adam"],
    "learning_rate": [5e-4, 1e-3, 5e-3, 1e-2],  # More reasonable range
    "hidden_dim": [32, 64, 128, 256],
    "num_layers": [1, 2, 4],
    "sentence_representation_type": ["last", "average", "max"],
}

In [ ]:
for hidden_dim in SEARCH_SPACE["hidden_dim"]:
    for num_layers in SEARCH_SPACE["num_layers"]:
        for optimizer_name in SEARCH_SPACE["optimizer_name"]:
            for batch_size in SEARCH_SPACE["batch_size"]:
                for learning_rate in SEARCH_SPACE["learning_rate"]:
                    for sentence_representation_type in SEARCH_SPACE["sentence_representation_type"]:
                        log_message = f"---------- batch_size_{batch_size}; lr_{learning_rate}; optimizer_{optimizer_name}; hidden_dim_{hidden_dim}; num_layers_{num_layers}; sentence_representation_{sentence_representation_type}  ----------"
                        print(log_message)
                        train_rnn_model_with_parameters(
                            embedding_matrix=embedding_matrix,
                            train_dataset=train_dataset,
                            val_dataset=valid_dataset,
                            batch_size=batch_size,
                            learning_rate=learning_rate,
                            optimizer_name=optimizer_name,
                            hidden_dim=hidden_dim,
                            num_layers=num_layers,
                            sentence_representation_type=sentence_representation_type,
                            show_progress=True,
                            freeze_embedding=False,
                        )

Seed set to 42


---------- batch_size_32; lr_0.0005; optimizer_RMSprop; hidden_dim_32; num_layers_1; sentence_representation_last  ----------


/home/linnsheng/Desktop/NTU/S3/Y1/NLP/SC4002/.venv/lib/python3.13/site-packages/lightning/pytorch/utilities/parsing.py:210: Attribute 'rnn_model' is an instance of `nn.Module` and is already saved during checkpointing. It is recommended to ignore them using `self.save_hyperparameters(ignore=['rnn_model'])`.
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name   | Type               | Params | Mode 
------------------------------------------------------
0 | model  | RNN                | 2.4 M  | train
1 | metric | MulticlassAccuracy | 0      | train
------------------------------------------------------
2.4 M     Trainable params
0         Non-trainable params
2.4 M     Total params
9.757     Total estimated model params size (MB)
9         Modules in train mode
0         Modules in eval mode


Epoch 18: 100%|██████████| 137/137 [00:03<00:00, 45.22it/s, v_num=0, train_loss=0.0145, train_acc=1.000, val_loss=2.350, val_acc=0.499] 


Seed set to 42


...

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name   | Type               | Params | Mode 
------------------------------------------------------
0 | model  | RNN                | 2.4 M  | train
1 | metric | MulticlassAccuracy | 0      | train
------------------------------------------------------
2.4 M     Trainable params
0         Non-trainable params
2.4 M     Total params
9.765     Total estimated model params size (MB)
9         Modules in train mode
0         Modules in eval mode


Epoch 5: 100%|██████████| 35/35 [00:00<00:00, 68.52it/s, v_num=0, train_loss=0.714, train_acc=0.854, val_loss=2.670, val_acc=0.365]


Seed set to 42


---------- batch_size_128; lr_0.005; optimizer_RMSprop; hidden_dim_32; num_layers_2; sentence_representation_average  ----------


GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name   | Type               | Params | Mode 
------------------------------------------------------
0 | model  | RNN                | 2.4 M  | train
1 | metric | MulticlassAccuracy | 0      | train
------------------------------------------------------
2.4 M     Trainable params
0         Non-trainable params
2.4 M     Total params
9.765     Total estimated model params size (MB)
9         Modules in train mode
0         Modules in eval mode


### Model Configuration Comparison

In [ ]:
train_results_df = load_tensorboard_logs(log_dir="tb_logs/rnn")

In [ ]:
rmsprop_sorted_df = train_results_df[train_results_df['optimizer_name'] == "RMSprop"].sort_values(by=["val_acc"], ascending=False).reset_index(drop=True)
rmsprop_sorted_df.head(n=50)

In [ ]:
train_results_df = train_results_df.sort_values(
    by=["val_acc"], ascending=False
).reset_index(drop=True)
train_results_df.head(20)

,val_acc,batch_size,hidden_dim,learning_rate,optimizer_name,train_loss,train_acc,num_layers,sentence_representation_type,freeze,epoch,val_loss,filename
0,0.741048,32,256,0.010,RMSprop,0.689391,0.753968,1,average,False,42.0,0.694172,events.out.tfevents.1762253482.ArchThinkpadT14...
1,0.739651,32,128,0.010,Adam,0.444329,0.882540,1,max,False,32.0,0.695786,events.out.tfevents.1762239394.ArchThinkpadT14...
2,0.739651,32,128,0.010,RMSprop,0.444329,0.882540,1,max,False,32.0,0.695786,events.out.tfevents.1762236381.ArchThinkpadT14...
3,0.721671,32,32,0.010,RMSprop,0.449705,0.885000,2,max,False,42.0,0.766004,events.out.tfevents.1762208799.ArchThinkpadT14...
4,0.721671,32,32,0.010,Adam,0.449705,0.885000,2,max,False,42.0,0.766004,events.out.tfevents.1762211207.ArchThinkpadT14...
5,0.718828,32,128,0.010,RMSprop,1.013844,0.502778,1,average,False,47.0,0.746652,events.out.tfevents.1762236229.ArchThinkpadT14...
6,0.718828,32,128,0.010,Adam,1.013844,0.502778,1,average,False,47.0,0.746652,events.out.tfevents.1762239250.ArchThinkpadT14...
7,0.706503,64,256,0.010,RMSprop,0.465816,0.732843,1,average,False,63.0,0.702203,events.out.tfevents.1762254632.ArchThinkpadT14...
8,0.701665,32,256,0.010,RMSprop,0.929195,0.769444,1,last,False,36.0,0.832590,events.out.tfevents.1762253362.ArchThinkpadT14...
9,0.698184,32,128,0.010,RMSprop,0.661537,0.711111,2,max,False,26.0,0.801680,events.out.tfevents.1762242385.ArchThinkpadT14...


### a) Report the final configuration of your best model, namely the number of training epochs, learning rate, optimizer, batch size and hidden dimension.

In [ ]:
train_results_df.head(1)

,val_acc,batch_size,hidden_dim,learning_rate,optimizer_name,train_loss,train_acc,num_layers,sentence_representation_type,freeze,epoch,val_loss,filename
0,0.741048,32,256,0.01,RMSprop,0.689391,0.753968,1,average,False,42.0,0.694172,events.out.tfevents.1762253482.ArchThinkpadT14...


Answer: Batch Size 32, Hidden Dimension 256, Learning Rate 0.01, RMSProp Optmizer and the number of training epochs is 42

### b) Report all the regularization strategies you have tried. Compare the accuracy on the test set among all strategies and the one without any regularization

In [ ]:
num_workers = os.cpu_count() // 2

train_dataloader = DataLoader(
    train_dataset,
    batch_size=32,
    shuffle=True,
    num_workers=num_workers,
    collate_fn=collate_fn,
    multiprocessing_context='spawn',
    persistent_workers=True,
)

valid_dataloader = DataLoader(
    valid_dataset,  
    batch_size=32,
    shuffle=False,
    num_workers=num_workers,
    collate_fn=collate_fn,
    multiprocessing_context='spawn',
    persistent_workers=True,
)

test_dataloader = DataLoader(  
    test_dataset,  
    batch_size=32,
    shuffle=False,
    num_workers=num_workers,
    collate_fn=collate_fn,
    multiprocessing_context='spawn',
    persistent_workers=True,
)

L.seed_everything(Config.SEED)

callbacks = [
    # EarlyStopping(
    #     monitor="val_loss",
    #     mode="min",
    #     patience=early_stopping_patience,
    #     min_delta=1e-4,
    # ),
    EarlyStopping(
        monitor="val_acc",
        mode="max",
        patience=3,
        min_delta=1e-4,
    ),
    ModelCheckpoint(
        monitor="val_loss",
        save_top_k=1,
        mode="min",
    ),
]

Seed set to 42


Without Regularization

In [ ]:
rnn_model = RNN(
    embedding_matrix=embedding_matrix,
    hidden_dim=256,
    num_layers=1,
    sentence_representation_type="average",
    output_dim=6,
    freeze_embedding=False,
)
model = RNNClassifier(rnn_model, lr=0.01, optimizer_name="RMSprop", show_progress=True)

/home/linnsheng/Desktop/NTU/S3/Y1/NLP/SC4002/.venv/lib/python3.13/site-packages/lightning/pytorch/utilities/parsing.py:210: Attribute 'rnn_model' is an instance of `nn.Module` and is already saved during checkpointing. It is recommended to ignore them using `self.save_hyperparameters(ignore=['rnn_model'])`.


In [ ]:
trainer = L.Trainer(
    max_epochs=50,
    accelerator="auto",
    gradient_clip_val=1.0,
)

trainer.fit(model, train_dataloaders=train_dataloader, val_dataloaders=valid_dataloader)

💡 Tip: For seamless cloud uploads and versioning, try installing [litmodels](https://pypi.org/project/litmodels/) to enable LitModelCheckpoint, which syncs automatically with the Lightning model registry.
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name   | Type               | Params | Mode 
------------------------------------------------------
0 | model  | RNN                | 2.6 M  | train
1 | metric | MulticlassAccuracy | 0      | train
------------------------------------------------------
2.6 M     Trainable params
0         Non-trainable params
2.6 M     Total params
10.418    Total estimated model params size (MB)
9         Modules in train mode
0         Modules in eval mode


Epoch 49: 100%|██████████| 110/110 [00:02<00:00, 44.52it/s, v_num=13, train_loss=0.490, train_acc=0.333, val_loss=0.735, val_acc=0.715] 

`Trainer.fit` stopped: `max_epochs=50` reached.


Epoch 49: 100%|██████████| 110/110 [00:02<00:00, 43.91it/s, v_num=13, train_loss=0.490, train_acc=0.333, val_loss=0.735, val_acc=0.715]


In [ ]:
trainer.test(model=model, dataloaders=test_dataloader)

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Testing DataLoader 0: 100%|██████████| 16/16 [00:00<00:00, 104.40it/s]


┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃        Test metric        ┃       DataLoader 0        ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│         test_acc          │    0.7517745494842529     │
│         test_loss         │    0.6915724873542786     │
└───────────────────────────┴───────────────────────────┘

[{'test_loss': 0.6915724873542786, 'test_acc': 0.7517745494842529}]

#### Early Stopping

In [ ]:
rnn_model = RNN(
    embedding_matrix=embedding_matrix,
    hidden_dim=256,
    num_layers=1,
    sentence_representation_type="average",
    output_dim=6,
    freeze_embedding=False,
)
model = RNNClassifier(rnn_model, lr=0.01, optimizer_name="RMSprop")

In [ ]:
trainer = L.Trainer(
    max_epochs=50,
    callbacks=callbacks,
    accelerator="auto",
    devices="auto",
    gradient_clip_val=1.0,
)

trainer.fit(model, train_dataloaders=train_dataloader, val_dataloaders=valid_dataloader)

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]



  | Name   | Type               | Params | Mode 
------------------------------------------------------
0 | model  | RNN                | 2.6 M  | train
1 | metric | MulticlassAccuracy | 0      | train
------------------------------------------------------
2.6 M     Trainable params
0         Non-trainable params
2.6 M     Total params
10.418    Total estimated model params size (MB)
9         Modules in train mode
0         Modules in eval mode


Epoch 49: 100%|██████████| 137/137 [00:02<00:00, 51.45it/s, v_num=8]        

`Trainer.fit` stopped: `max_epochs=50` reached.


Epoch 49: 100%|██████████| 137/137 [00:02<00:00, 51.35it/s, v_num=8]


In [ ]:
trainer.test(model=model, dataloaders=test_dataloader)

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Testing DataLoader 0: 100%|██████████| 16/16 [00:00<00:00, 148.04it/s]


┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃        Test metric        ┃       DataLoader 0        ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│         test_acc          │    0.7604537606239319     │
│         test_loss         │    0.6336861252784729     │
└───────────────────────────┴───────────────────────────┘

[{'test_loss': 0.6336861252784729, 'test_acc': 0.7604537606239319}]

### L2 + Early Stopping

In [ ]:
rnn_model = RNN(
    embedding_matrix=embedding_matrix,
    hidden_dim=256,
    num_layers=1,
    sentence_representation_type="average",
    output_dim=6,
    freeze_embedding=False,
)
model = RNNClassifier(rnn_model, lr=0.01, optimizer_name="RMSprop", weight_decay=1e-5, show_progress=True)

In [ ]:
trainer = L.Trainer(
    max_epochs=50,
    callbacks=callbacks,
    accelerator="auto",
    devices="auto",
)

trainer.fit(model, train_dataloaders=train_dataloader, val_dataloaders=valid_dataloader)

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name   | Type               | Params | Mode 
------------------------------------------------------
0 | model  | RNN                | 2.6 M  | train
1 | metric | MulticlassAccuracy | 0      | train
------------------------------------------------------
2.6 M     Trainable params
0         Non-trainable params
2.6 M     Total params
10.418    Total estimated model params size (MB)
9         Modules in train mode
0         Modules in eval mode


Epoch 21: 100%|██████████| 110/110 [00:02<00:00, 41.82it/s, v_num=14, train_loss=2.540, train_acc=0.000, val_loss=2.260, val_acc=0.270]


In [ ]:
trainer.test(model=model, dataloaders=test_dataloader)

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Testing DataLoader 0: 100%|██████████| 16/16 [00:00<00:00, 128.46it/s]


┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃        Test metric        ┃       DataLoader 0        ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│         test_acc          │    0.2334180325269699     │
│         test_loss         │    2.2802817821502686     │
└───────────────────────────┴───────────────────────────┘

[{'test_loss': 2.2802817821502686, 'test_acc': 0.2334180325269699}]

### DropOut + Early Stopping

In [ ]:
rnn_model = RNN(
    embedding_matrix=embedding_matrix,
    hidden_dim=256,
    num_layers=1,
    sentence_representation_type="average",
    output_dim=6,
    freeze_embedding=False,
    dropout=0.5,
    embedding_dropout=0.2,
)
model = RNNClassifier(rnn_model, lr=0.01, optimizer_name="RMSprop")

In [ ]:
trainer = L.Trainer(
    max_epochs=50,
    callbacks=callbacks,
    accelerator="auto",
    devices="auto",
)

trainer.fit(model, train_dataloaders=train_dataloader, val_dataloaders=valid_dataloader)

Trainer already configured with model summary callbacks: [<class 'lightning.pytorch.callbacks.model_summary.ModelSummary'>]. Skipping setting a default `ModelSummary` callback.
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
/home/linnsheng/Desktop/NTU/S3/Y1/NLP/SC4002/.venv/lib/python3.13/site-packages/lightning/pytorch/callbacks/model_checkpoint.py:751: Checkpoint directory /home/linnsheng/Desktop/NTU/S3/Y1/NLP/SC4002/lightning_logs/version_14/checkpoints exists and is not empty.
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]



  | Name   | Type               | Params | Mode 
------------------------------------------------------
0 | model  | RNN                | 2.6 M  | train
1 | metric | MulticlassAccuracy | 0      | train
------------------------------------------------------
2.6 M     Trainable params
0         Non-trainable params
2.6 M     Total params
10.418    Total estimated model params size (MB)
9         Modules in train mode
0         Modules in eval mode


Epoch 0: 100%|██████████| 110/110 [00:09<00:00, 11.58it/s, v_num=15]       


In [ ]:
trainer.test(model=model, dataloaders=test_dataloader)

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Testing DataLoader 0: 100%|██████████| 16/16 [00:00<00:00, 85.10it/s]


┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃        Test metric        ┃       DataLoader 0        ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│         test_acc          │    0.18720002472400665    │
│         test_loss         │    1.7479004859924316     │
└───────────────────────────┴───────────────────────────┘

[{'test_loss': 1.7479004859924316, 'test_acc': 0.18720002472400665}]

### L2 + DropOut + Early Stopping

In [ ]:
rnn_model = RNN(
    embedding_matrix=embedding_matrix,
    hidden_dim=256,
    num_layers=1,
    sentence_representation_type="average",
    output_dim=6,
    freeze_embedding=False,
    dropout=0.5,
    embedding_dropout=0.2,
)
model = RNNClassifier(rnn_model, lr=0.01, optimizer_name="RMSprop", weight_decay=1e-4)

In [ ]:
trainer = L.Trainer(
    max_epochs=50,
    callbacks=callbacks,
    accelerator="auto",
    devices="auto",
)
trainer.fit(model, train_dataloaders=train_dataloader, val_dataloaders=valid_dataloader)

Trainer already configured with model summary callbacks: [<class 'lightning.pytorch.callbacks.model_summary.ModelSummary'>]. Skipping setting a default `ModelSummary` callback.
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name   | Type               | Params | Mode 
------------------------------------------------------
0 | model  | RNN                | 2.6 M  | train
1 | metric | MulticlassAccuracy | 0      | train
------------------------------------------------------
2.6 M     Trainable params
0         Non-trainable params
2.6 M     Total params
10.418    Total estimated model params size (MB)
9         Modules in train mode
0         Modules in eval mode


Epoch 0: 100%|██████████| 110/110 [00:02<00:00, 53.77it/s, v_num=16]        


In [ ]:
trainer.test(model=model, dataloaders=test_dataloader)

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Testing DataLoader 0: 100%|██████████| 16/16 [00:00<00:00, 93.94it/s] 


┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃        Test metric        ┃       DataLoader 0        ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│         test_acc          │    0.34235653281211853    │
│         test_loss         │    1.7192174196243286     │
└───────────────────────────┴───────────────────────────┘

[{'test_loss': 1.7192174196243286, 'test_acc': 0.34235653281211853}]